In [ ]:
# Notebook to preprocess the datasets and create new datasets required for the analysis.
# import required packages
import numpy as np
import pandas as pd

drivers=pd.read_csv('.//data//drivers.csv')
drivers.head()

champresults=pd.read_csv('.//data//driver_standings.csv')
qualiresults=pd.read_csv('.//data//qualifying.csv')
raceresults=pd.read_csv('.//data//races.csv')